# DIPY Analysis Example

This notebook demonstrates diffusion MRI analysis using DIPY with sample data and visualizations.

## Setup and Installation

First, let's install the required packages:

In [ ]:
# Install required packages
!pip install dipy matplotlib nibabel numpy scipy plotly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import json
from datetime import datetime

# DIPY imports
from dipy.data import get_fnames
from dipy.io.image import load_nifti, save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
from dipy.segment.mask import median_otsu
from dipy.viz import regtools

print("✅ All packages imported successfully!")
print(f"Analysis started at: {datetime.now()}")

## Load Sample Data

We'll use DIPY's built-in sample dataset for this demonstration:

In [ ]:
# Load sample DWI data
hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')

data, affine = load_nifti(hardi_fname)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

print(f"Data shape: {data.shape}")
print(f"Number of volumes: {data.shape[-1]}")
print(f"Number of b-values: {len(np.unique(bvals))}")
print(f"B-values: {np.unique(bvals)}")

## Brain Extraction with Median OTSU

Let's perform skull stripping using the median OTSU algorithm:

In [ ]:
# Perform brain extraction
b0_mask, mask = median_otsu(data, vol_idx=range(10, 50), median_radius=3, numpass=1)

print("✅ Brain extraction completed!")
print(f"Original data shape: {data.shape}")
print(f"Mask shape: {mask.shape}")
print(f"Brain voxels: {np.sum(mask)}")
print(f"Total voxels: {np.prod(mask.shape)}")
print(f"Brain volume ratio: {np.sum(mask)/np.prod(mask.shape):.2%}")

## Visualization 1: Brain Mask Overlay

Let's visualize the brain extraction results:

In [ ]:
# Create brain mask visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Brain Extraction Results', fontsize=16, fontweight='bold')

# Select middle slices
slice_x = data.shape[0] // 2
slice_y = data.shape[1] // 2  
slice_z = data.shape[2] // 2

# Original images
axes[0, 0].imshow(data[slice_x, :, :, 0].T, cmap='gray', origin='lower')
axes[0, 0].set_title('Original - Sagittal')
axes[0, 0].axis('off')

axes[0, 1].imshow(data[:, slice_y, :, 0].T, cmap='gray', origin='lower')
axes[0, 1].set_title('Original - Coronal')
axes[0, 1].axis('off')

axes[0, 2].imshow(data[:, :, slice_z, 0].T, cmap='gray', origin='lower')
axes[0, 2].set_title('Original - Axial')
axes[0, 2].axis('off')

# Brain extracted images
brain_data = data * mask[..., np.newaxis]

axes[1, 0].imshow(brain_data[slice_x, :, :, 0].T, cmap='gray', origin='lower')
axes[1, 0].set_title('Brain Extracted - Sagittal')
axes[1, 0].axis('off')

axes[1, 1].imshow(brain_data[:, slice_y, :, 0].T, cmap='gray', origin='lower')
axes[1, 1].set_title('Brain Extracted - Coronal')
axes[1, 1].axis('off')

axes[1, 2].imshow(brain_data[:, :, slice_z, 0].T, cmap='gray', origin='lower')
axes[1, 2].set_title('Brain Extracted - Axial')
axes[1, 2].axis('off')

plt.tight_layout()
plt.savefig('brain_extraction_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Brain extraction visualization saved as 'brain_extraction_results.png'")

## DTI Analysis

Now let's fit the diffusion tensor model and compute DTI metrics:

In [ ]:
# Fit DTI model
tenmodel = TensorModel(gtab)
tenfit = tenmodel.fit(data, mask=mask)

# Compute DTI metrics
fa = tenfit.fa
md = tenfit.md
ad = tenfit.ad
rd = tenfit.rd

print("✅ DTI analysis completed!")
print(f"FA range: {fa[mask].min():.3f} - {fa[mask].max():.3f}")
print(f"MD range: {md[mask].min():.6f} - {md[mask].max():.6f}")
print(f"AD range: {ad[mask].min():.6f} - {ad[mask].max():.6f}")
print(f"RD range: {rd[mask].min():.6f} - {rd[mask].max():.6f}")

## Visualization 2: DTI Metrics Maps

Let's create colorful maps of the DTI metrics:

In [ ]:
# Create DTI metrics visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
fig.suptitle('DTI Metrics - Axial Slice', fontsize=16, fontweight='bold')

slice_z = data.shape[2] // 2

# FA map
fa_slice = fa[:, :, slice_z]
fa_slice[~mask[:, :, slice_z]] = 0
im1 = axes[0, 0].imshow(fa_slice.T, cmap='hot', origin='lower', vmin=0, vmax=1)
axes[0, 0].set_title('Fractional Anisotropy (FA)')
axes[0, 0].axis('off')
plt.colorbar(im1, ax=axes[0, 0], fraction=0.046, pad=0.04)

# MD map
md_slice = md[:, :, slice_z]
md_slice[~mask[:, :, slice_z]] = 0
im2 = axes[0, 1].imshow(md_slice.T, cmap='viridis', origin='lower')
axes[0, 1].set_title('Mean Diffusivity (MD)')
axes[0, 1].axis('off')
plt.colorbar(im2, ax=axes[0, 1], fraction=0.046, pad=0.04)

# AD map
ad_slice = ad[:, :, slice_z]
ad_slice[~mask[:, :, slice_z]] = 0
im3 = axes[1, 0].imshow(ad_slice.T, cmap='plasma', origin='lower')
axes[1, 0].set_title('Axial Diffusivity (AD)')
axes[1, 0].axis('off')
plt.colorbar(im3, ax=axes[1, 0], fraction=0.046, pad=0.04)

# RD map
rd_slice = rd[:, :, slice_z]
rd_slice[~mask[:, :, slice_z]] = 0
im4 = axes[1, 1].imshow(rd_slice.T, cmap='coolwarm', origin='lower')
axes[1, 1].set_title('Radial Diffusivity (RD)')
axes[1, 1].axis('off')
plt.colorbar(im4, ax=axes[1, 1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.savefig('dti_metrics_maps.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 DTI metrics maps saved as 'dti_metrics_maps.png'")

## Visualization 3: Interactive Histogram Analysis

Let's create interactive histograms of the DTI metrics:

In [ ]:
# Create interactive histograms
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Fractional Anisotropy', 'Mean Diffusivity', 'Axial Diffusivity', 'Radial Diffusivity'],
    vertical_spacing=0.12
)

# FA histogram
fig.add_trace(
    go.Histogram(
        x=fa[mask], 
        nbinsx=50, 
        name='FA',
        marker_color='red',
        opacity=0.7
    ),
    row=1, col=1
)

# MD histogram
fig.add_trace(
    go.Histogram(
        x=md[mask], 
        nbinsx=50, 
        name='MD',
        marker_color='blue',
        opacity=0.7
    ),
    row=1, col=2
)

# AD histogram
fig.add_trace(
    go.Histogram(
        x=ad[mask], 
        nbinsx=50, 
        name='AD',
        marker_color='green',
        opacity=0.7
    ),
    row=2, col=1
)

# RD histogram
fig.add_trace(
    go.Histogram(
        x=rd[mask], 
        nbinsx=50, 
        name='RD',
        marker_color='orange',
        opacity=0.7
    ),
    row=2, col=2
)

fig.update_layout(
    title_text='DTI Metrics Histograms',
    title_x=0.5,
    height=600,
    showlegend=False
)

# Update axes labels
fig.update_xaxes(title_text="FA Value", row=1, col=1)
fig.update_xaxes(title_text="MD Value", row=1, col=2)
fig.update_xaxes(title_text="AD Value", row=2, col=1)
fig.update_xaxes(title_text="RD Value", row=2, col=2)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Count", row=2, col=2)

fig.write_html('dti_histograms.html')
fig.show()

print("📊 Interactive DTI histograms saved as 'dti_histograms.html'")

## Visualization 4: 3D Brain Volume

Let's create a 3D visualization of the brain volume:

In [ ]:
# Create 3D brain volume visualization
# Sample the volume for performance
step = 4
x, y, z = np.mgrid[0:mask.shape[0]:step, 0:mask.shape[1]:step, 0:mask.shape[2]:step]
mask_sampled = mask[::step, ::step, ::step]
fa_sampled = fa[::step, ::step, ::step]

# Create 3D scatter plot
brain_coords = np.where(mask_sampled)
fa_values = fa_sampled[brain_coords]

fig = go.Figure(data=go.Scatter3d(
    x=brain_coords[0],
    y=brain_coords[1], 
    z=brain_coords[2],
    mode='markers',
    marker=dict(
        size=2,
        color=fa_values,
        colorscale='Hot',
        opacity=0.6,
        colorbar=dict(title="FA Value")
    ),
    text=fa_values,
    hovertemplate='<b>FA:</b> %{text:.3f}<extra></extra>'
))

fig.update_layout(
    title='3D Brain Volume (FA-colored)',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        )
    ),
    width=800,
    height=600
)

fig.write_html('brain_3d_volume.html')
fig.show()

print("📊 3D brain volume visualization saved as 'brain_3d_volume.html'")

## Analysis Summary

Let's create a summary of our analysis:

In [ ]:
# Create analysis summary
summary = {
    'analysis_info': {
        'timestamp': datetime.now().isoformat(),
        'data_shape': data.shape,
        'num_volumes': data.shape[-1],
        'num_bvalues': len(np.unique(bvals)),
        'bvalues': np.unique(bvals).tolist()
    },
    'brain_extraction': {
        'total_voxels': int(np.prod(mask.shape)),
        'brain_voxels': int(np.sum(mask)),
        'brain_volume_ratio': float(np.sum(mask)/np.prod(mask.shape))
    },
    'dti_metrics': {
        'fa': {
            'min': float(fa[mask].min()),
            'max': float(fa[mask].max()),
            'mean': float(fa[mask].mean()),
            'std': float(fa[mask].std())
        },
        'md': {
            'min': float(md[mask].min()),
            'max': float(md[mask].max()),
            'mean': float(md[mask].mean()),
            'std': float(md[mask].std())
        },
        'ad': {
            'min': float(ad[mask].min()),
            'max': float(ad[mask].max()),
            'mean': float(ad[mask].mean()),
            'std': float(ad[mask].std())
        },
        'rd': {
            'min': float(rd[mask].min()),
            'max': float(rd[mask].max()),
            'mean': float(rd[mask].mean()),
            'std': float(rd[mask].std())
        }
    },
    'generated_files': [
        'brain_extraction_results.png',
        'dti_metrics_maps.png', 
        'dti_histograms.html',
        'brain_3d_volume.html'
    ]
}

# Save summary as JSON
with open('analysis_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

# Display summary
print("="*60)
print("📋 ANALYSIS SUMMARY")
print("="*60)
print(f"Analysis completed at: {summary['analysis_info']['timestamp']}")
print(f"Data shape: {summary['analysis_info']['data_shape']}")
print(f"Brain volume ratio: {summary['brain_extraction']['brain_volume_ratio']:.2%}")
print(f"\nDTI Metrics (mean ± std):")
print(f"  FA: {summary['dti_metrics']['fa']['mean']:.3f} ± {summary['dti_metrics']['fa']['std']:.3f}")
print(f"  MD: {summary['dti_metrics']['md']['mean']:.6f} ± {summary['dti_metrics']['md']['std']:.6f}")
print(f"  AD: {summary['dti_metrics']['ad']['mean']:.6f} ± {summary['dti_metrics']['ad']['std']:.6f}")
print(f"  RD: {summary['dti_metrics']['rd']['mean']:.6f} ± {summary['dti_metrics']['rd']['std']:.6f}")
print(f"\nGenerated files:")
for file in summary['generated_files']:
    print(f"  📄 {file}")
print("="*60)
print("✅ Analysis complete! All visualizations saved.")

## Conclusion

This notebook demonstrates a complete DIPY analysis pipeline including:

1. **Data Loading**: Loading diffusion MRI data with b-values and b-vectors
2. **Brain Extraction**: Using median OTSU algorithm for skull stripping
3. **DTI Analysis**: Fitting tensor model and computing metrics (FA, MD, AD, RD)
4. **Visualizations**: 
   - Static images showing brain extraction results
   - Colorful DTI metrics maps
   - Interactive histograms of DTI metrics
   - 3D brain volume visualization

All visualizations and analysis results are saved as files that can be shared or published.

---

*Generated with DIPY analysis pipeline - NeuroHackademy 2025*